This notebook analyses a subsample of a narrow definition of creative 
which has been subsequently tagged and then looks at a definition based on analysing words likely
to minimise false positives

In [1]:
import pandas as pd
import numpy as np
import re

1. Examines the UK subsample from the narrow definition match which has been manually annotated with a flag =1 if considered creative

In [2]:
arx_uk=pd.read_csv("filepath/data//creative_subset_new_uk.csv", dtype={'article_id':str})


In [3]:
arx_uk

,Flag,Unnamed: 0,article_id,article_created,article_updated,article_title,journal_ref,doi,summary,citation_count,...,ai,kw_n_advertising_and_marketing,kw_n_architecture,kw_n_immersive_games,kw_n_design,"kw_n_film,_tv,_video,_radio_and_photography","kw_n_museums,_galleries_and_libraries","kw_n_music,_performing_and_visual_arts",kw_n_publishing,year
0,NaN,4218,802.1296,2/9/2008,5/13/2008,On quantum statistics in data analysis,NaN,NaN,"Originally, quantum probability theory was d...",5,...,True,0,0,0,0,0,0,0,1,2008
1,1.0,11925,904.3351,4/21/2009,NaN,A Subsequence-Histogram Method for Generic Voc...,NaN,NaN,We consider the problem of recognizing a voc...,0,...,True,1,0,0,0,0,0,0,0,2009
2,1.0,13137,906.4026,6/22/2009,6/23/2009,A Quantum-based Model for Interactive Informat...,NaN,NaN,Even the best information retrieval model ca...,5,...,True,1,0,0,0,0,0,0,0,2009
3,1.0,19279,1002.2353,2/11/2010,NaN,Fighting Online Click-Fraud Using Bluff Ads,NaN,NaN,Online advertising is currently the greatest...,63,...,True,1,0,0,0,0,0,0,0,2010
4,NaN,20312,1003.414,3/22/2010,NaN,Integrating Real-Time Analysis With The Dendri...,Proceedings of Genetic and Evolutionary Comput...,NaN,"As an immune inspired algorithm, the Dendrit...",20,...,True,0,0,0,0,0,0,1,0,2010
5,NaN,21047,1004.2468,4/14/2010,NaN,Quantum learning: optimal classification of qu...,New J. Phys. 12 123032 (2010),10.1088/1367-2630/12/12/123032,Pattern recognition is a central topic in Le...,12,...,True,0,0,0,0,0,0,1,0,2010
6,1.0,23746,1006.4948,6/25/2010,NaN,Automatic Music Composition using Answer Set P...,NaN,NaN,Music composition used to be a pen and paper...,19,...,True,0,0,0,0,0,0,1,0,2010
7,1.0,28334,1011.3241,11/14/2010,NaN,New Methods of Analysis of Narrative and Seman...,"Entertainment Computing, 2, 115-121, 2011",10.1016/j.entcom.2010.12.008,Our work has focused on support for film or ...,7,...,True,0,0,0,0,2,0,0,1,2010
8,NaN,40065,1109.2136,9/9/2011,NaN,Learning Content Selection Rules for Generatin...,"Journal Of Artificial Intelligence Research, V...",10.1613/jair.1591,A fundamental requirement of any task-orient...,74,...,True,0,0,0,0,0,0,1,0,2011
9,NaN,50839,1204.4107,4/18/2012,2/15/2016,Towards the Evolution of Vertical-Axis Wind Tu...,"Evolutionary Intelligence (2014), 7(3):155-167",10.1007/s12065-014-0116-4,We have recently presented an initial study ...,11,...,True,0,0,0,0,0,0,0,1,2012


In [4]:
list(arx_uk)

['Flag',
 'Unnamed: 0',
 'article_id',
 'article_created',
 'article_updated',
 'article_title',
 'journal_ref',
 'doi',
 'summary',
 'citation_count',
 'citation_count_updated',
 'arxiv_categories',
 'matching_score',
 'is_multinational',
 'institute_id',
 'institute_name',
 'institute_city',
 'institute_state',
 'institute_country',
 'institute_lat',
 'institute_lon',
 'ai',
 'kw_n_advertising_and_marketing',
 'kw_n_architecture',
 'kw_n_immersive_games',
 'kw_n_design',
 'kw_n_film,_tv,_video,_radio_and_photography',
 'kw_n_museums,_galleries_and_libraries',
 'kw_n_music,_performing_and_visual_arts',
 'kw_n_publishing',
 'year']

In [5]:
arx_uk['Flag'].sum()

48.0

In [6]:
arx_uk['Flag'].shape

(132,)

In [7]:
#Replace the nans with 0
arx_uk['Flag']=arx_uk['Flag'].fillna(0)

In [8]:
creative_categories=['kw_n_advertising_and_marketing','kw_n_architecture',
 'kw_n_immersive_games','kw_n_design','kw_n_film,_tv,_video,_radio_and_photography',
 'kw_n_museums,_galleries_and_libraries','kw_n_music,_performing_and_visual_arts',
 'kw_n_publishing']

table = pd.pivot_table(arx_uk, index=['Flag'], values=creative_categories, aggfunc=np.sum)

#Converts it into a dataframe
table= pd.DataFrame(table)
table

,kw_n_advertising_and_marketing,kw_n_architecture,kw_n_design,"kw_n_film,_tv,_video,_radio_and_photography",kw_n_immersive_games,"kw_n_museums,_galleries_and_libraries","kw_n_music,_performing_and_visual_arts",kw_n_publishing
Flag,,,,,,,,
0.0,4,3,6,8,5,12,40,9
1.0,9,0,1,10,5,4,22,6


Most of these categories are not very successful a lot of false positives

2. Looks at the whole dataset with a set of key words that are unlikely to lead to false positives

In [9]:
arx=pd.read_csv("filepath//data/processed//deep_learning.csv", dtype={'article_id':str})


Adds in a flag for AI status

In [10]:
#File that contains the AI ids that are matched to the arx article_ids

ai_path = 'filepath//data//dl_paper_ids.csv'

ml_ids = pd.read_csv(ai_path,dtype={'paper_id':str})

ml_ids.shape

ml_ids_set = set(list(ml_ids.loc[ml_ids['is_AI']==True,'paper_id']))


arx['ai'] = [x in ml_ids_set for x in arx['article_id']]

A series of functions that match various creative terms that are less likely to lead to false positives

In [11]:
def vfxsearch(x):
 vfxcategory=['visual effects','vfx', 'visual_effects','visual-effects','visualeffects']
 regex = re.compile("|".join(word for word in vfxcategory), re.IGNORECASE)
 if regex.search(x['summary'] or x['title'] ):
    return 1
 else:
    return 0


def gamesearch(x):
 gamecategory=['computer games', 'video games', 'video-games', 'computer-games', 'mmorpg', 'nintendo', 'playstation']
 regex = re.compile("|".join(word for word in gamecategory), re.IGNORECASE)
 if regex.search(x['summary'] or x['title'] ):
    return 1
 else:
    return 0

def immersivesearch(x):
 immersivecategory=['virtual reality', 'augmented reality',  r"\bar\b",  r"\bar\b", 'virtual-reality', 
                    'augmented-reality', "Oculus Rift","Magic Leap"]
 regex = re.compile("|".join(word for word in immersivecategory), re.IGNORECASE)
 if regex.search(x['summary'] or x['title'] ):
    return 1
 else:
    return 0

def adsearch(x):
 adcategory=['advertising', 'adverts']
 regex = re.compile("|".join(word for word in adcategory), re.IGNORECASE)
 if regex.search(x['summary'] or x['title'] ):
    return 1
 else:
    return 0

def artsearch(x):
 artcategory=['artistic','creative']
 regex = re.compile("|".join(word for word in artcategory), re.IGNORECASE)
 if regex.search(x['summary'] or x['title'] ):
    return 1
 else:
    return 0

Applies these functions

In [12]:
#List of the functions to loop through
art_categories=[immersivesearch, adsearch, gamesearch, artsearch, vfxsearch]

#Applies the functions
for  elem in art_categories: #The _name_ extracts the function name
   arx[elem.__name__]=arx.apply(elem, axis=1)  


In [13]:
list(arx)

['Unnamed: 0',
 'article_id',
 'article_created',
 'article_updated',
 'article_title',
 'journal_ref',
 'doi',
 'summary',
 'citation_count',
 'citation_count_updated',
 'arxiv_categories',
 'matching_score',
 'is_multinational',
 'institute_id',
 'institute_name',
 'institute_city',
 'institute_state',
 'institute_country',
 'institute_lat',
 'institute_lon',
 'ai',
 'immersivesearch',
 'adsearch',
 'gamesearch',
 'artsearch',
 'vfxsearch']

In [14]:
#Creates a category that is not captured by the others

arx['other']=np.where(((arx['vfxsearch']==0) & (arx['adsearch']==0) & (arx['immersivesearch']==0) & (arx['gamesearch']==0) & (arx['artsearch']==0)), 1,0)

In [15]:
#Turns function names into list
create_categories=[elem.__name__ for elem in art_categories]
create_categories 

['immersivesearch', 'adsearch', 'gamesearch', 'artsearch', 'vfxsearch']

 Does a table splitting it by the AI variable

In [16]:
ai_table = pd.pivot_table(arx, index=['ai'], values=create_categories, aggfunc=np.sum)

#Converts it into a dataframe
ai_table= pd.DataFrame(ai_table)
ai_table

,adsearch,artsearch,gamesearch,immersivesearch,vfxsearch
ai,,,,,
False,795,625,198,1027,29
True,359,344,119,384,17


Does a table for AI==True splitting it by the countries

In [17]:
by_country = pd.pivot_table(arx[arx['ai']==True], index=['institute_country'], values=create_categories, aggfunc=np.sum)

#Converts it into a dataframe
table_country= pd.DataFrame(by_country)
table_country.sort_values(by=['immersivesearch'], ascending=False).head(n=10)

,adsearch,artsearch,gamesearch,immersivesearch,vfxsearch
institute_country,,,,,
United States,74,68,23,54,6
China,26,14,2,29,3
United Kingdom,30,24,9,23,1
France,11,11,2,15,0
Germany,10,9,1,15,0
Australia,15,14,2,11,0
Israel,9,9,2,10,0
Netherlands,11,4,1,9,0
India,11,5,1,9,0
